In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pywaffle import Waffle

import bokeh

# The following lines are helpful to make bokeh plots show up in 
from bokeh.resources import INLINE
import bokeh.io
from bokeh import *
bokeh.io.output_notebook(INLINE)
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure, output_file, show
from bokeh.transform import dodge
from bokeh.io import export_svg, export_png

import random
from datetime import datetime
from datetime import timedelta
from dateutil import rrule
from bokeh.io import output_notebook
output_notebook()

import numpy as np

# plot = Plot(output_backend="svg")
# option two
# plot.output_backend = "svg"

# Figures for A and B Pictogram

In [ ]:
fig = plt.figure(
    FigureClass=Waffle, 
    rows=5, 
    columns=10,
    values=[40, 40, 20],
    colors = ["#789DB7","#001b48", "#72bcd4"],
    figsize=(100, 40)
)
# plt.show()
fig.savefig("pictogram_anno.svg", format='svg', dpi=1200)

In [ ]:
fig = plt.figure(
    FigureClass=Waffle, 
    rows=5, 
    columns=10,
    values=[51, 38, 11],
    colors = ["#789DB7","#001b48", "#72bcd4"],
    figsize=(100, 40)
)
# plt.show()
fig.savefig("pictogram_full.svg", format='svg', dpi=1200)

# Figures for Annotated dataset

In [ ]:
david_full = pd.read_csv("sarah_anno_pol_only.csv")
sarah_col="New_Label"
print(david_full[sarah_col].value_counts())

david_full['date_day'] = david_full['Datetime'].apply(lambda x: pd.Timestamp(x).isocalendar()[2])
david_full['date_week'] = david_full['Datetime'].apply(lambda x: pd.Timestamp(x).isocalendar()[1])
david_full['date_year'] = david_full['Datetime'].apply(lambda x: pd.Timestamp(x).isocalendar()[0])
david_full['month'] = david_full['Datetime'].apply(lambda x: pd.Timestamp(x).month_name())
david_full['day'] = david_full['Datetime'].apply(lambda x: pd.Timestamp(x).day)
david_full['year'] = david_full['Datetime'].apply(lambda x: pd.Timestamp(x).year)
david_full['week'] = david_full['Datetime'].apply(lambda x: pd.Timestamp(x).week)
enc_party = {'Azad Samaj Party':'ASP','Bhagwa Kranti Sena':'BKS','Rashtriya Janta Dal':'RJD',
             'Rashtriya Lok Dal':'RLD','Shiromani Akali Dal':'SAD','Azad Samaj Partyno-':'ASP'}

david_full['Party'] = david_full['Party'].apply(lambda x: enc_party[x] if x in enc_party else x)

week=david_full["week"]
new_labels=david_full[sarah_col]
assert len(week)==len(new_labels)
dict_week_p_full=[0]*12
dict_week_t_full=[0]*12
dict_week_n_full=[0]*12
for i,j in zip(week,new_labels):
    if i==52 or i==13:
        continue
    elif j=="T":
        dict_week_t_full[i-1]+=1
    elif j=="P":
        dict_week_p_full[i-1]+=1
    elif j=="N":
        dict_week_n_full[i-1]+=1

assert len(dict_week_p_full)==len(dict_week_t_full)==len(dict_week_n_full)
print(len(dict_week_p_full))

party_dict={"N":{},"P":{},"T":{}}
for user_type in david_full['Post'].unique():
    if user_type == "Party Handle": 
        user_type = "Party"
    elif user_type == "Politician" or user_type == "Political Leader":
        user_type = "Politician"
    party_dict["N"][user_type]=0.0
    party_dict["P"][user_type]=0.0
    party_dict["T"][user_type]=0.0
new_labels = david_full[sarah_col]
poster = david_full["Post"]
for user_type, j in zip(poster,new_labels):
    if user_type == "Party Handle": 
        user_type = "Party"
    elif user_type == "Politician" or user_type == "Political Leader":
        user_type = "Politician"
    party_dict[j][user_type]+=1
print(party_dict)

for label in party_dict:
    data = party_dict[label]
    total = sum(data.values())
    per = {k:(v*100)/total for k,v in data.items()}
    party_dict[label]=per
print(party_dict)

In [ ]:
week_int = list(range(1,13))
week = [str(i) for i in week_int]
x = np.asarray(week_int)
y = np.asarray([n+p+t for n,p,t in zip(dict_week_n_full,dict_week_p_full,dict_week_t_full)])
print(y)
p = figure(x_range=(0.7,13),
    frame_width=900,
    frame_height=450,
    output_backend="svg"
)

p.line(x, y, line_color="black", line_width=4)
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, Scatter
source = ColumnDataSource(dict(x=x, y=y))
glyph = Scatter(x="x", y="y", size=30, marker="circle",fill_color="black")
p.add_glyph(source, glyph)
# p.scatter(x, y,,fill_alpha=1,)
p.xaxis.ticker = x
p.yaxis.ticker = [100+i*100 for i in range(0,4)]
p.xgrid.visible = False
p.ygrid.visible = False
p.min_border_left = 0
p.min_border_right = 0
p.min_border_top = 0
p.min_border_bottom = 0

p.xaxis.axis_label = 'Week'
p.yaxis.axis_label = 'Number of posts'
p.xaxis.major_label_text_font_size = "30pt"
p.yaxis.major_label_text_font_size = "30pt"
p.xaxis.axis_label_text_font_size = "30pt"
p.yaxis.axis_label_text_font_size = "30pt"

p.xaxis.axis_label_text_font_style = 'bold'
p.yaxis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_text_font_style = "bold"
p.yaxis.major_label_text_font_style = "bold"
p.xaxis.axis_line_width = 4
p.yaxis.axis_line_width = 4
p.yaxis.major_label_text_color = "black"
p.xaxis.major_label_text_color = "black"
p.xaxis.axis_label_text_color = "black"
p.yaxis.axis_label_text_color = "black"
p.xaxis.major_label_standoff=2
p.yaxis.major_label_standoff=2
p.xaxis.major_tick_line_alpha=0
p.yaxis.major_tick_line_alpha=0
p.xaxis.axis_label_standoff = 1
p.yaxis.axis_label_standoff = 1
p.yaxis.axis_label_text_font = "calibri"
p.xaxis.axis_label_text_font = "calibri"
p.xaxis.major_label_text_font = "calibri"
p.yaxis.major_label_text_font = "calibri"

p.outline_line_color=None ## Remove top and right border
# p.xaxis.axis_label_text_font_style = 'calibri'
show(p)

export_svg(p, filename="all_posts_anno_pol.svg")
# export_png(p, filename="all_posts_anno.png")

In [ ]:
week_int = list(range(1,13))
week = [str(i) for i in week_int]
x = np.asarray(week_int)
vals = ['N','P','T']
my_data = {'week':week,
   'N':dict_week_n_full,
   'P':dict_week_p_full,
   'T':dict_week_t_full
}
p = figure(x_range = week, frame_width=900,
    frame_height=450,
           y_range=(0, 130),output_backend="svg")
colors = ["#789DB7","#001b48", "#72bcd4"]
p.vbar(x = dodge('week', -0.25, range = p.x_range), top = 'N',
width = 0.2,source = my_data, color = colors[0])
p.vbar(x = dodge('week', 0.0, range = p.x_range), top = 'P',
width = 0.2, source = my_data,color = colors[1]) 
p.vbar(x = dodge('week', 0.25, range = p.x_range), top = 'T',
width = 0.2,source = my_data,color = colors[2])
# p.xaxis.ticker = x
p.yaxis.ticker = [30+i*30 for i in range(0,4)]
# p.yaxis.ticker = [100+i*100 for i in range(0,5)]
p.xgrid.visible = False
p.ygrid.visible = False
p.min_border_left = 0
p.min_border_right = 0
p.min_border_top = 0
p.min_border_bottom = 0

p.xaxis.axis_label = 'Week'
p.yaxis.axis_label = 'Number of posts'
p.xaxis.major_label_text_font_size = "30pt"
p.yaxis.major_label_text_font_size = "30pt"
p.xaxis.axis_label_text_font_size = "30pt"
p.yaxis.axis_label_text_font_size = "30pt"

p.xaxis.axis_label_text_font_style = 'bold'
p.yaxis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_text_font_style = "bold"
p.yaxis.major_label_text_font_style = "bold"
p.xaxis.axis_line_width = 4
p.yaxis.axis_line_width = 4
p.yaxis.major_label_text_color = "black"
p.xaxis.major_label_text_color = "black"
p.xaxis.axis_label_text_color = "black"
p.yaxis.axis_label_text_color = "black"
p.xaxis.major_label_standoff=2
p.yaxis.major_label_standoff=2
p.xaxis.major_tick_line_alpha=0
p.yaxis.major_tick_line_alpha=0
p.xaxis.axis_label_standoff = 1
p.yaxis.axis_label_standoff = 1
p.yaxis.axis_label_text_font = "calibri"
p.xaxis.axis_label_text_font = "calibri"
p.xaxis.major_label_text_font = "calibri"
p.yaxis.major_label_text_font = "calibri"

p.outline_line_color=None ## Remove top and right border
show(p)

export_svg(p, filename="n_posts_anno_pol.svg")

# Figures for Full dataset

In [ ]:
david_full = pd.read_csv("sarah_all_pol_only.csv")
print(david_full.prediction.value_counts())

david_full['date_day'] = david_full['Datetime'].apply(lambda x: pd.Timestamp(x).isocalendar()[2])
david_full['date_week'] = david_full['Datetime'].apply(lambda x: pd.Timestamp(x).isocalendar()[1])
david_full['date_year'] = david_full['Datetime'].apply(lambda x: pd.Timestamp(x).isocalendar()[0])
david_full['month'] = david_full['Datetime'].apply(lambda x: pd.Timestamp(x).month_name())
david_full['day'] = david_full['Datetime'].apply(lambda x: pd.Timestamp(x).day)
david_full['year'] = david_full['Datetime'].apply(lambda x: pd.Timestamp(x).year)
david_full['week'] = david_full['Datetime'].apply(lambda x: pd.Timestamp(x).week)
enc_party = {'Azad Samaj Party':'ASP','Bhagwa Kranti Sena':'BKS','Rashtriya Janta Dal':'RJD',
             'Rashtriya Lok Dal':'RLD','Shiromani Akali Dal':'SAD','Azad Samaj Partyno-':'ASP'}

david_full['Party'] = david_full['Party'].apply(lambda x: enc_party[x] if x in enc_party else x)

sarah_col="prediction"
week=david_full["week"]
new_labels=david_full[sarah_col]
assert len(week)==len(new_labels)
dict_week_p_full=[0]*12
dict_week_t_full=[0]*12
dict_week_n_full=[0]*12
for i,j in zip(week,new_labels):
    if i==52 or i==13:
        continue
    elif j==1:
        dict_week_t_full[i-1]+=1
    elif j==2:
        dict_week_p_full[i-1]+=1
    elif j==0:
        dict_week_n_full[i-1]+=1

assert len(dict_week_p_full)==len(dict_week_t_full)==len(dict_week_n_full)
print(len(dict_week_p_full))


party_dict={1:{},2:{},0:{}}
for user_type in david_full['Post'].unique():
    if user_type == "Party Handle": 
        user_type = "Party"
    elif user_type == "Politician" or user_type == "Political Leader":
        user_type = "Politician"
    party_dict[1][user_type]=0.0
    party_dict[2][user_type]=0.0
    party_dict[0][user_type]=0.0
new_labels = david_full[sarah_col]
poster = david_full["Post"]
for user_type, j in zip(poster,new_labels):
    if user_type == "Party Handle": 
        user_type = "Party"
    elif user_type == "Politician" or user_type == "Political Leader":
        user_type = "Politician"
    party_dict[j][user_type]+=1
print(party_dict)

for label in party_dict:
    data = party_dict[label]
    total = sum(data.values())
    per = {k:(v*100)/total for k,v in data.items()}
    party_dict[label]=per
print(party_dict)

In [ ]:
x = np.asarray(week_int)
y = np.asarray([n+p+t for n,p,t in zip(dict_week_n_full,dict_week_p_full,dict_week_t_full)])
print(y)
p = figure(x_range=(0.7,13), 
#     frame_width=1000,
#     frame_height=650,
    frame_width=900,
    frame_height=450,
    output_backend="svg"
)

p.line(x, y, line_color="black", line_width=4)
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, Scatter
source = ColumnDataSource(dict(x=x, y=y))
glyph = Scatter(x="x", y="y", size=30, marker="circle",fill_color="black")
p.add_glyph(source, glyph)
p.xaxis.ticker = x
p.yaxis.ticker = [(i+0.5)*1000 for i in range(1,7)]
p.xgrid.visible = False
p.ygrid.visible = False
p.min_border_left = 0
p.min_border_right = 0
p.min_border_top = 0
p.min_border_bottom = 0

p.xaxis.axis_label = 'Week'
p.yaxis.axis_label = 'Number of posts'
p.xaxis.major_label_text_font_size = "30pt"
p.yaxis.major_label_text_font_size = "30pt"
p.xaxis.axis_label_text_font_size = "30pt"
p.yaxis.axis_label_text_font_size = "30pt"

p.xaxis.axis_label_text_font_style = 'bold'
p.yaxis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_text_font_style = "bold"
p.yaxis.major_label_text_font_style = "bold"
p.xaxis.axis_line_width = 4
p.yaxis.axis_line_width = 4
p.yaxis.major_label_text_color = "black"
p.xaxis.major_label_text_color = "black"
p.xaxis.axis_label_text_color = "black"
p.yaxis.axis_label_text_color = "black"
p.xaxis.major_label_standoff=2
p.yaxis.major_label_standoff=2
p.xaxis.major_tick_line_alpha=0
p.yaxis.major_tick_line_alpha=0
p.xaxis.axis_label_standoff = 1
p.yaxis.axis_label_standoff = 1
p.yaxis.axis_label_text_font = "calibri"
p.xaxis.axis_label_text_font = "calibri"
p.xaxis.major_label_text_font = "calibri"
p.yaxis.major_label_text_font = "calibri"

p.outline_line_color=None ## Remove top and right border
show(p)
export_svg(p, filename="all_posts_full_pol.svg")
# export_png(p, filename="all_posts_full.png")

In [ ]:
week_int = list(range(1,13))
week = [str(i) for i in week_int]
vals = ['N','P','T']
my_data = {'week':week,
   'N':dict_week_n_full,
   'P':dict_week_p_full,
   'T':dict_week_t_full
}
colors = ["#789DB7","#001b48", "#72bcd4"]
p = figure(x_range = week, 
               frame_width=900,
    frame_height=450,
           y_range=(0,3000),
           output_backend="svg")
p.vbar(x = dodge('week', -0.25, range = p.x_range), top = 'N',
width = 0.2,source = my_data, color = colors[0])
p.vbar(x = dodge('week', 0.0, range = p.x_range), top = 'P',
width = 0.2, source = my_data,color = colors[1]) 
#          "#17A7B8")
p.vbar(x = dodge('week', 0.25, range = p.x_range), top = 'T',
width = 0.2,source = my_data,color = colors[2])
#          "#718dbf")
# p.xaxis.ticker = x
p.yaxis.ticker = [i*500 for i in range(0,6)]
# show(p)
p.xgrid.visible = False
p.ygrid.visible = False
p.min_border_left = 0
p.min_border_right = 0
p.min_border_top = 0
p.min_border_bottom = 0

p.xaxis.axis_label = 'Week'
p.yaxis.axis_label = 'Number of posts'
p.xaxis.major_label_text_font_size = "30pt"
p.yaxis.major_label_text_font_size = "30pt"
p.xaxis.axis_label_text_font_size = "30pt"
p.yaxis.axis_label_text_font_size = "30pt"

p.xaxis.axis_label_text_font_style = 'bold'
p.yaxis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_text_font_style = "bold"
p.yaxis.major_label_text_font_style = "bold"
p.xaxis.axis_line_width = 4
p.yaxis.axis_line_width = 4
p.yaxis.major_label_text_color = "black"
p.xaxis.major_label_text_color = "black"
p.xaxis.axis_label_text_color = "black"
p.yaxis.axis_label_text_color = "black"
p.xaxis.major_label_standoff=2
p.yaxis.major_label_standoff=2
p.xaxis.major_tick_line_alpha=0
p.yaxis.major_tick_line_alpha=0
p.xaxis.axis_label_standoff = 1
p.yaxis.axis_label_standoff = 1
p.yaxis.axis_label_text_font = "calibri"
p.xaxis.axis_label_text_font = "calibri"
p.xaxis.major_label_text_font = "calibri"
p.yaxis.major_label_text_font = "calibri"

p.outline_line_color=None
show(p)

export_svg(p, filename="n_posts_full_pol.svg")
# export_png(p, filename="n_posts_full.png")

In [ ]:
from mpl_chord_diagram import chord_diagram
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

flux_per = np.array([[332, 140, 9, 0],
        [406, 12, 5, 2, ],
        [ 93, 29, 45, 0, ],
        [240, 0, 0, 290, ]])
# flux = flux*100
names = ['BJP', 'INC', 'AAP', 'SP']


grad = False
gap = 0.1
sorts = "distance"
nrota = False
cmaps = None
fclrs = "black"                                    # fontcolors
drctd = True
pad = 15
cmaps = ["#CC97DD",
         "#D4638D",
         "#ADACAC",
         "#8B864E"
#          "#9CCB19"
        ]
fig, ax = plt.subplots(figsize=(40, 40))
chord_diagram(flux, names=None, gap=gap, use_gradient=grad, sort=sorts, directed=drctd,
              colors = cmaps,
              rotate_names=nrota, fontcolor=fclrs, pad = pad,width=0.4,
              ax=ax,fontsize=40,
             )
plt.show()
# # plt.savefig("chord_anno.svg",format="svg",dpi=1200)
# # plt.savefig("chord_anno.png",format="png")